# Game of life on PYNQ to demonstrate annimation

Code derived from example here:
    
http://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
from pynq.lib.video import *
#mode = VideoMode(1280, 720, 8)
width = 640
height = 480
mode = VideoMode(width, height, 8)

hdmi_out = base.video.hdmi_out
hdmi_out.configure(mode, PIXEL_GRAY)
hdmi_out.start()

In [3]:
import numpy as np
np.random.seed(0)

def life_step_1(X):
    """Game of life step using generator expressions"""
    nbrs_count = sum(np.roll(np.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0))
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

def life_step_2(X):
    """Game of life step using scipy tools"""
    from scipy.signal import convolve2d
    nbrs_count = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))
    
life_step = life_step_1

In [7]:
X = np.zeros((height, width), dtype=bool)

def life_animation(X):

    outframe = hdmi_out.newframe()   
    outframe[:] = X
    r = np.random.random((height, width))
    X[:] = (r > 0.75)

    X = np.asarray(X)
    assert X.ndim == 2
    X = X.astype(bool)

    X_blank = np.zeros_like(X)

    # animation function.  This is called sequentially
    def animate(i):
        im.set_data(animate.X)
        animate.X = life_step(animate.X)
        return (im,)
    animate.X = X

    outframe[:] = X[:]*0xff

    hdmi_out.writeframe(outframe)

# Game of life step using generator expressions

In [8]:
life_step = life_step_1

In [9]:
import time
start = time.time()
numframes = 20
for i in range(numframes):
    life_animation(X)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Frames per second:  14.856946722011925


# Game of life step using scipy tools

In [10]:
life_step = life_step_2

In [11]:
import time
start = time.time()
numframes = 20
for i in range(numframes):
    life_animation(X)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Frames per second:  14.983028324230094


# Single step

In [15]:
life_animation(X)